<a href="https://colab.research.google.com/github/rachelyayra/object-detectors/blob/main/Feature_extractions_for_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deepface

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import dlib
import cmake
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import pickle
from deepface.commons import functions
from sklearn.metrics.pairwise import euclidean_distances
import os

In [ ]:
#creating a database
student_list = []
folderpath = "/content/gdrive/MyDrive/student_database"
listOfpics = os.listdir(folderpath)
labels = []
#print(listOfpics)
for pic in listOfpics:
  filepath = os.path.join(folderpath,pic)
  #new_pic = cv2.imread(filepath)
  student_list.append(filepath)
  label = pic[0:pic.index('.')]
  labels.append(label)
#print(labels)
print(labels)

['Selorm', 'Ten Lee', 'Kwon Soonyoung', 'Lee Jihoon', 'Xu Minghao']


In [ ]:
e_model = cv2.dnn.readNetFromTorch("/content/gdrive/MyDrive/nn4.small2.v1.t7")
target_size = (96,96)
#img_path1 = "/content/gdrive/MyDrive/facepics/ten1.jpg"
img_path2 = "/content/gdrive/MyDrive/facepics/babypic"


In [ ]:
#encode
#preprocess list
student_embedding = []

for student in student_list:
  img1 = functions.preprocess_face(student,target_size = target_size)
  img1_blob = cv2.dnn.blobFromImage(img1[0])
  e_model.setInput(img1_blob)
  img1_embedded = e_model.forward()
  student_embedding.append(img1_embedded)


In [ ]:
#img1 = functions.preprocess_face(img_path1,target_size = target_size)
img2 = functions.preprocess_face(img_path2,target_size = target_size)
display_img = cv2.imread(img_path2,cv2.IMREAD_COLOR)

<class 'numpy.ndarray'>


In [ ]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    xmin = rect.left()
    ymin = rect.top()
    w = rect.right() - xmin
    h = rect.bottom() - ymin

    # return a tuple of (x, y, w, h)
    return (xmin, ymin, w, h)

In [ ]:
#face_rect ,d_img,xs,ys = detectFace(display_img)

In [ ]:
#img1_blob = cv2.dnn.blobFromImage(img1[0])
img2_blob = cv2.dnn.blobFromImage(img2[0])
print(img1_blob.shape)

(1, 3, 96, 96)


In [ ]:
#e_model.setInput(img1_blob)
#img1_embedded = e_model.forward()
#print(img1_embedded)
e_model.setInput(img2_blob)
img2_embedded = e_model.forward()
#print(img2_embedded)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
dist_list = []
for i in student_embedding:
  distance = euclidean_distances(i,img2_embedded)[0,0]
  dist_list.append(distance)
  

In [ ]:
#print(dist_list)
threshold = 0.60 
bool_list = []
for i in dist_list:
  if i > 0.60:
    bol = True
    bool_list.append(bol)
  else:
    bol = False
    bool_list.append(bol)
#print(bool_list)
  
if (all(bool_list)):
  print('no')
else:
  reg_face = labels[dist_list.index(min(dist_list))]
  print(reg_face)

Selorm


In [ ]:
def detectFace(image,regface):
  detector = dlib.get_frontal_face_detector()
  face = detector(image)
  numberOfFaces = len(face) 
  listOfbb = []
  for rect in face:
    listOfbb.append(rect_to_bb(rect))

  color = (255, 0, 0)
  x_values = []
  y_values = []
  for num in listOfbb:
    #print(num)
    img = cv2.rectangle(image,num,color)
    im_with_text = cv2.putText(image, regface ,  num[0:2], cv2.FONT_HERSHEY_COMPLEX, 1, color, 1)
    x_values.append(num[0]) 
    y_values.append(num[1])
  #print("There are "+str(numberOfFaces)+ " faces")
  return face ,im_with_text,x_values,y_values

In [ ]:
face_rect ,n_img,xs,ys = detectFace(display_img,reg_face)
cv2_imshow(display_img)